In [1]:
import tensorflow as tf
from tensorboard.plugins.hparams import api as hp
import matplotlib.pylab as plt
import numpy as np
import pandas as pd


In [2]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()
x_train = x_train / 255.
x_test = x_test / 255.

train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(64)
valid_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(64)

In [3]:
next(iter(train_ds))[1].shape

TensorShape([64])

In [4]:
HP_NUM_UNITS = hp.HParam("num_units", hp.Discrete([16, 32, 64, 128]))
HP_DROPOUT = hp.HParam("dropout", hp.RealInterval(.1, .5))
HP_OPTIMIZER = hp.HParam("optimizer", hp.Discrete(["adam", "nadam", "sgd", "rmsprop"]))
HPARAMS = [HP_NUM_UNITS, HP_DROPOUT, HP_OPTIMIZER]

    
METRIC_ACCURACY_MEAN = hp.Metric("accuracy mean", display_name="Accuracy mean")
METRIC_ACCURACY_MAX = hp.Metric("accuracy max", display_name="Accuracy max")
METRIC_ACCURACY_MIN = hp.Metric("accuracy min", display_name="Accuracy min")
METRICS = [METRIC_ACCURACY_MEAN, METRIC_ACCURACY_MAX, METRIC_ACCURACY_MIN]

PARENT_DIR = "logs/TensorBoard_hyperparameters/"

with tf.summary.create_file_writer(PARENT_DIR).as_default():
    hp.hparams_config(
        hparams = HPARAMS,
        metrics = METRICS,
    )

In [5]:
EPOCHS = 1

def train_test_model(session_num, hparams):
    model = tf.keras.models.Sequential([
        tf.keras.layers.Input(shape=[28,28]),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(hparams[HP_NUM_UNITS], activation="relu"),
        tf.keras.layers.Dropout(hparams[HP_DROPOUT]),
        tf.keras.layers.Dense(10, activation="softmax")
    ])

    model.compile(loss="sparse_categorical_crossentropy", optimizer=hparams[HP_OPTIMIZER], metrics=["accuracy"])
    
    history = model.fit(train_ds, epochs=10, validation_data=(valid_ds), callbacks=[tf.keras.callbacks.TensorBoard(PARENT_DIR + "run-" + str(session_num))])
#     print(history.history)
    return history.history

In [6]:
def record_and_run(session_num, hparams):
    with tf.summary.create_file_writer(PARENT_DIR + "run-" + str(session_num)).as_default():
        hp.hparams(hparams)
        history = train_test_model(session_num, hparams)
        tf.summary.scalar("accuracy mean", tf.squeeze(tf.reduce_mean(history["val_accuracy"])), step=session_num)
        tf.summary.scalar("accuracy max", tf.squeeze(tf.reduce_max(history["val_accuracy"])), step=session_num)
        tf.summary.scalar("accuracy min", tf.squeeze(tf.reduce_min(history["val_accuracy"])), step=session_num)


In [7]:
session_num = 0

for num_units in HP_NUM_UNITS.domain.values:
    for dropout_rate in np.linspace(HP_DROPOUT.domain.min_value, HP_DROPOUT.domain.max_value, 5):
        for optimizer in HP_OPTIMIZER.domain.values:
            hparams = {
                HP_NUM_UNITS: num_units,
                HP_DROPOUT: dropout_rate,
                HP_OPTIMIZER: optimizer,
            }
            
            run_name = "run-%d" % session_num
            print("--- Session ", session_num)
            print({h.name:hparams[h] for h in hparams})
            record_and_run(session_num, hparams)
            session_num += 1

--- Session  0
{'num_units': 16, 'dropout': 0.1, 'optimizer': 'adam'}
Epoch 1/10
  1/938 [..............................] - ETA: 0s - loss: 2.3963 - accuracy: 0.0938WARNING:tensorflow:From C:\Users\ikuchin\AppData\Local\Continuum\miniconda3\envs\tf23\lib\site-packages\tensorflow\python\ops\summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
938/938 [==============================] - 2s 2ms/step - loss: 0.8306 - accuracy: 0.7085 - val_loss: 0.5559 - val_accuracy: 0.8052
Epoch 2/10
938/938 [==============================] - 2s 2ms/step - loss: 0.5734 - accuracy: 0.7953 - val_loss: 0.4914 - val_accuracy: 0.8255
Epoch 3/10
938/938 [==============================] - 2s 2ms/step - loss: 0.5251 - accuracy: 0.8095 - val_loss: 0.4669 - val_accuracy: 0.8313
Epoch 4/10
938/938 [==============================] - 2s 2ms/step - loss: 0.4963 - accuracy: 0.8203 -

Epoch 1/10
938/938 [==============================] - 2s 2ms/step - loss: 0.9188 - accuracy: 0.6629 - val_loss: 0.5556 - val_accuracy: 0.8022
Epoch 2/10
938/938 [==============================] - 2s 2ms/step - loss: 0.6391 - accuracy: 0.7661 - val_loss: 0.4953 - val_accuracy: 0.8282
Epoch 3/10
938/938 [==============================] - 2s 2ms/step - loss: 0.5888 - accuracy: 0.7834 - val_loss: 0.4662 - val_accuracy: 0.8384
Epoch 4/10
938/938 [==============================] - 2s 2ms/step - loss: 0.5407 - accuracy: 0.8037 - val_loss: 0.4492 - val_accuracy: 0.8398
Epoch 5/10
938/938 [==============================] - 2s 2ms/step - loss: 0.5215 - accuracy: 0.8107 - val_loss: 0.4438 - val_accuracy: 0.8440
Epoch 6/10
938/938 [==============================] - 2s 2ms/step - loss: 0.5056 - accuracy: 0.8172 - val_loss: 0.4410 - val_accuracy: 0.8438
Epoch 7/10
938/938 [==============================] - 2s 2ms/step - loss: 0.4915 - accuracy: 0.8209 - val_loss: 0.4301 - val_accuracy: 0.8482
Epoch 

938/938 [==============================] - 2s 2ms/step - loss: 0.9396 - accuracy: 0.6685 - val_loss: 0.5491 - val_accuracy: 0.8065
Epoch 2/10
938/938 [==============================] - 2s 2ms/step - loss: 0.6759 - accuracy: 0.7578 - val_loss: 0.4959 - val_accuracy: 0.8238
Epoch 3/10
938/938 [==============================] - 2s 2ms/step - loss: 0.6309 - accuracy: 0.7717 - val_loss: 0.4770 - val_accuracy: 0.8286
Epoch 4/10
938/938 [==============================] - 2s 2ms/step - loss: 0.6082 - accuracy: 0.7786 - val_loss: 0.4610 - val_accuracy: 0.8365
Epoch 5/10
938/938 [==============================] - 2s 2ms/step - loss: 0.5908 - accuracy: 0.7831 - val_loss: 0.4555 - val_accuracy: 0.8409
Epoch 6/10
938/938 [==============================] - 2s 2ms/step - loss: 0.5765 - accuracy: 0.7889 - val_loss: 0.4535 - val_accuracy: 0.8380
Epoch 7/10
938/938 [==============================] - 2s 2ms/step - loss: 0.5749 - accuracy: 0.7889 - val_loss: 0.4479 - val_accuracy: 0.8405
Epoch 8/10
938/93

Epoch 4/10
938/938 [==============================] - 2s 2ms/step - loss: 0.8129 - accuracy: 0.6948 - val_loss: 0.5335 - val_accuracy: 0.8131
Epoch 5/10
938/938 [==============================] - 2s 2ms/step - loss: 0.7994 - accuracy: 0.6979 - val_loss: 0.5303 - val_accuracy: 0.8180
Epoch 6/10
938/938 [==============================] - 2s 2ms/step - loss: 0.7892 - accuracy: 0.7041 - val_loss: 0.5207 - val_accuracy: 0.8186
Epoch 7/10
938/938 [==============================] - 2s 2ms/step - loss: 0.7776 - accuracy: 0.7067 - val_loss: 0.5180 - val_accuracy: 0.8224
Epoch 8/10
938/938 [==============================] - 2s 2ms/step - loss: 0.7601 - accuracy: 0.7133 - val_loss: 0.5186 - val_accuracy: 0.8208
Epoch 9/10
938/938 [==============================] - 2s 2ms/step - loss: 0.7352 - accuracy: 0.7219 - val_loss: 0.5022 - val_accuracy: 0.8227
Epoch 10/10
938/938 [==============================] - 2s 2ms/step - loss: 0.7131 - accuracy: 0.7292 - val_loss: 0.4895 - val_accuracy: 0.8280
--- S

938/938 [==============================] - 2s 2ms/step - loss: 0.9955 - accuracy: 0.6269 - val_loss: 0.5941 - val_accuracy: 0.8171
Epoch 8/10
938/938 [==============================] - 2s 2ms/step - loss: 0.9856 - accuracy: 0.6297 - val_loss: 0.5786 - val_accuracy: 0.8229
Epoch 9/10
938/938 [==============================] - 2s 2ms/step - loss: 0.9724 - accuracy: 0.6365 - val_loss: 0.5825 - val_accuracy: 0.8209
Epoch 10/10
938/938 [==============================] - 2s 2ms/step - loss: 0.9712 - accuracy: 0.6378 - val_loss: 0.5814 - val_accuracy: 0.8224
--- Session  17
{'num_units': 16, 'dropout': 0.5, 'optimizer': 'nadam'}
Epoch 1/10
938/938 [==============================] - 2s 2ms/step - loss: 1.2570 - accuracy: 0.5293 - val_loss: 0.6604 - val_accuracy: 0.7798
Epoch 2/10
938/938 [==============================] - 2s 2ms/step - loss: 0.9187 - accuracy: 0.6440 - val_loss: 0.5760 - val_accuracy: 0.7938
Epoch 3/10
938/938 [==============================] - 2s 2ms/step - loss: 0.8709 - acc

...

938/938 [==============================] - 4s 5ms/step - loss: 1.1835 - accuracy: 0.5987 - val_loss: 0.7413 - val_accuracy: 0.7508
Epoch 2/10
938/938 [==============================] - 3s 3ms/step - loss: 0.8019 - accuracy: 0.7294 - val_loss: 0.6259 - val_accuracy: 0.7895
Epoch 3/10
938/938 [==============================] - 4s 4ms/step - loss: 0.7077 - accuracy: 0.7646 - val_loss: 0.5732 - val_accuracy: 0.8052
Epoch 4/10
938/938 [==============================] - 3s 3ms/step - loss: 0.6555 - accuracy: 0.7790 - val_loss: 0.5400 - val_accuracy: 0.8122
Epoch 5/10
938/938 [==============================] - 4s 4ms/step - loss: 0.6209 - accuracy: 0.7917 - val_loss: 0.5180 - val_accuracy: 0.8201
Epoch 6/10
938/938 [==============================] - 3s 3ms/step - loss: 0.5916 - accuracy: 0.7994 - val_loss: 0.5013 - val_accuracy: 0.8218
Epoch 7/10
938/938 [==============================] - 4s 4ms/step - loss: 0.5756 - accuracy: 0.8044 - val_loss: 0.4887 - val_accuracy: 0.8252
Epoch 8/10
938/93

938/938 [==============================] - 4s 4ms/step - loss: 0.5493 - accuracy: 0.8156 - val_loss: 0.5291 - val_accuracy: 0.8205
Epoch 4/10
938/938 [==============================] - 4s 4ms/step - loss: 0.5135 - accuracy: 0.8258 - val_loss: 0.5045 - val_accuracy: 0.8261
Epoch 5/10
938/938 [==============================] - 3s 4ms/step - loss: 0.4911 - accuracy: 0.8318 - val_loss: 0.4871 - val_accuracy: 0.8326
Epoch 6/10
938/938 [==============================] - 4s 4ms/step - loss: 0.4723 - accuracy: 0.8382 - val_loss: 0.4732 - val_accuracy: 0.8361
Epoch 7/10
938/938 [==============================] - 4s 5ms/step - loss: 0.4583 - accuracy: 0.8426 - val_loss: 0.4627 - val_accuracy: 0.8383
Epoch 8/10
938/938 [==============================] - 4s 5ms/step - loss: 0.4445 - accuracy: 0.8474 - val_loss: 0.4531 - val_accuracy: 0.8407
Epoch 9/10
938/938 [==============================] - 4s 4ms/step - loss: 0.4340 - accuracy: 0.8509 - val_loss: 0.4450 - val_accuracy: 0.8435
Epoch 10/10
938/9

938/938 [==============================] - 3s 4ms/step - loss: 0.4861 - accuracy: 0.8336 - val_loss: 0.4771 - val_accuracy: 0.8341
Epoch 7/10
938/938 [==============================] - 3s 4ms/step - loss: 0.4695 - accuracy: 0.8367 - val_loss: 0.4661 - val_accuracy: 0.8378
Epoch 8/10
938/938 [==============================] - 3s 4ms/step - loss: 0.4567 - accuracy: 0.8410 - val_loss: 0.4549 - val_accuracy: 0.8416
Epoch 9/10
938/938 [==============================] - 4s 4ms/step - loss: 0.4462 - accuracy: 0.8462 - val_loss: 0.4476 - val_accuracy: 0.8416
Epoch 10/10
938/938 [==============================] - 3s 3ms/step - loss: 0.4359 - accuracy: 0.8489 - val_loss: 0.4402 - val_accuracy: 0.8431
--- Session  68
{'num_units': 128, 'dropout': 0.30000000000000004, 'optimizer': 'adam'}
Epoch 1/10
938/938 [==============================] - 4s 4ms/step - loss: 0.5769 - accuracy: 0.7979 - val_loss: 0.4385 - val_accuracy: 0.8416
Epoch 2/10
938/938 [==============================] - 3s 4ms/step - lo

Epoch 9/10
938/938 [==============================] - 4s 4ms/step - loss: 0.4591 - accuracy: 0.8406 - val_loss: 0.4500 - val_accuracy: 0.8418
Epoch 10/10
938/938 [==============================] - 4s 4ms/step - loss: 0.4505 - accuracy: 0.8437 - val_loss: 0.4434 - val_accuracy: 0.8434
--- Session  72
{'num_units': 128, 'dropout': 0.4, 'optimizer': 'adam'}
Epoch 1/10
938/938 [==============================] - 5s 5ms/step - loss: 0.6116 - accuracy: 0.7873 - val_loss: 0.4547 - val_accuracy: 0.8353
Epoch 2/10
938/938 [==============================] - 4s 4ms/step - loss: 0.4473 - accuracy: 0.8396 - val_loss: 0.4109 - val_accuracy: 0.8513
Epoch 3/10
938/938 [==============================] - 4s 5ms/step - loss: 0.4106 - accuracy: 0.8509 - val_loss: 0.3869 - val_accuracy: 0.8598
Epoch 4/10
938/938 [==============================] - 4s 5ms/step - loss: 0.3879 - accuracy: 0.8576 - val_loss: 0.3761 - val_accuracy: 0.8623
Epoch 5/10
938/938 [==============================] - 4s 4ms/step - loss: 0

938/938 [==============================] - 4s 4ms/step - loss: 0.6443 - accuracy: 0.7721 - val_loss: 0.4611 - val_accuracy: 0.8336
Epoch 2/10
938/938 [==============================] - 4s 4ms/step - loss: 0.4758 - accuracy: 0.8306 - val_loss: 0.4233 - val_accuracy: 0.8474
Epoch 3/10
938/938 [==============================] - 3s 4ms/step - loss: 0.4378 - accuracy: 0.8426 - val_loss: 0.3980 - val_accuracy: 0.8554
Epoch 4/10
938/938 [==============================] - 3s 4ms/step - loss: 0.4171 - accuracy: 0.8500 - val_loss: 0.3893 - val_accuracy: 0.8613
Epoch 5/10
938/938 [==============================] - 3s 3ms/step - loss: 0.4013 - accuracy: 0.8547 - val_loss: 0.3717 - val_accuracy: 0.8667
Epoch 6/10
938/938 [==============================] - 4s 4ms/step - loss: 0.3919 - accuracy: 0.8582 - val_loss: 0.3757 - val_accuracy: 0.8636
Epoch 7/10
938/938 [==============================] - 3s 4ms/step - loss: 0.3809 - accuracy: 0.8613 - val_loss: 0.3716 - val_accuracy: 0.8693
Epoch 8/10
938/93